In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time

#binary = FirefoxBinary('path/to/binary')
browser = webdriver.Chrome(executable_path= "C:\chromedriver.exe")
browser.get("https://www.stockbiz.vn/IndicesStats.aspx")

target_page = 1

# Display time started
print(time.strftime("%H:%M:%S", time.localtime()))

# Run main web scraping function
full_scrape(target_page)

# Display time concluded
print(time.strftime("%H:%M:%S", time.localtime()))

# elem = browser.find_element_by_xpath('//*[@id="ctl00_webPartManager_wp267165551_wp1192412521_callbackData"]/div')
# # button = browser.find_element_by_class_name("pageNavigation")
# # print(button)
# #button.click()
# #time.sleep(5)

# elem.click()
# time.sleep(0.2)

# elem = browser.find_element_by_xpath("//*")
# print(button.get_attribute("outerHTML"))

def gen_hcp_dict():
    WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, "//div[@class='table-head']")))
    hcp_name =  driver.find_element_by_xpath("//div[@class='table-head']").text    
    all_fields = driver.find_elements_by_xpath("//td[@class='no-border table-data']") # Using find elementS since there are multiple elements    
    hcp_data = []
    # Extract test for every field within the table
    for field in all_fields:
        hcp_data.append(field.text)
    hcp_dict = {}
    # Assign extracted data into respective key-value pairs
    hcp_dict['name'] = hcp_name
    hcp_dict['reg_number'] = hcp_data[0]
    # hcp_data[1] is just a blank space, so it can be ignored
    hcp_dict['reg_date'] = hcp_data[2]
    hcp_dict['reg_end_date'] = hcp_data[3]
    hcp_dict['reg_type'] = hcp_data[4]
    hcp_dict['practice_status'] = hcp_data[5]
    hcp_dict['cert_start_date'] = hcp_data[6]
    hcp_dict['cert_end_date'] = hcp_data[7]
    hcp_dict['qualification'] = hcp_data[8]
    hcp_dict['practice_place_name'] = hcp_data[9]
    hcp_dict['practice_place_address'] = hcp_data[10]
    hcp_dict['practice_place_phone'] = hcp_data[11]
    
    return hcp_dict
#==========================================================================================================#

def get_absolute_last_page():
    # Find all elements with pagination class (since it contains 
    # page numbers)
    WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, "//a[@class='pagination']")))
    all_pages = driver.find_elements_by_xpath("//a[@class='pagination']")

    # Get final element, which corresponds to 'Last' hyperlink
    # (which will go to last page number)
    last_elem = all_pages[-1].get_attribute('href')

    # Keep only the number of last page
    last_page_num = int(re.sub("[^0-9]", "", last_elem))
    
    return last_page_num
 #==========================================================================================================#   
def get_current_pagination_range():
    WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, "//a[@class='pagination']")))
    # Get all elements related to pagination
    all_pages = driver.find_elements_by_xpath("//a[@class='pagination']")
    # Implicit wait to allow page to load
    driver.implicitly_wait(1)
    # Find numbers of pagination range, and append to list
    pagination_range_on_page = []
    for elem in all_pages:
        # Only extracting numeric values of pagination range
        if elem.text.isnumeric():
            pagination_range_on_page.append(int(elem.text))
            driver.implicitly_wait(1)
        else:
            pass
    driver.implicitly_wait(1)
    return pagination_range_on_page
#==========================================================================================================#
def locate_target_page(target_page):
    last_page_num = get_absolute_last_page()
    # Find midway point of all search results page
    midway_point = last_page_num/2
    # If target page is in first half, start clicking from the start
    if target_page < midway_point: 
        current_page_num = get_current_page()
        if current_page_num == target_page:
            pass
        else:            
            pagination_range = get_current_pagination_range()
            while target_page not in pagination_range:
                driver.implicitly_wait(1)
               # If target page is not in pagination range, keep 
               # clicking last pagination number to go down the list
                click_last_pagination_num(pagination_range) 
                current_page_num = get_current_page()
                pagination_range = get_current_pagination_range()
                driver.implicitly_wait(1)
            else:
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.LINK_TEXT, f"{target_page}"))).click() # Once target          page is in pagination page, go to the target page
    # If target page is in later half of list, go to Last page and      
    # move in reverse (This saves alot of time)
    else: 
        WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.LINK_TEXT, 'Last'))).click()
        time.sleep(sleeptime)
        current_page_num = get_current_page()
        if current_page_num == target_page:
            pass
        else:           
            pagination_range = get_current_pagination_range()
            while target_page not in pagination_range:
                driver.implicitly_wait(2)
                click_first_pagination_num(pagination_range)
                current_page_num = get_current_page()
                pagination_range = get_current_pagination_range()
            else:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, f"{target_page}"))).click() 
                # Once target page is in pagination page, go to the target page
#==========================================================================================================#
def full_scrape(target_page):

    last_page_num = get_absolute_last_page()
    driver.implicitly_wait(1)
    
    while target_page != last_page_num:
        locate_target_page(target_page)
        print('Starting with target page ' + str(target_page))
        
        # Retrieve HTML from search page
        target_page_html = driver.find_element_by_xpath("//body").get_attribute('outerHTML')
        driver.implicitly_wait(1)
        
        # Find list of all IDs on that page, and keep the unique IDs
        all_ids = re.findall("P[0-9]{5}[A-Z]{1}", target_page_html)
        id_list = list(dict.fromkeys(all_ids))

        for index, hcp_id in enumerate(id_list): # Tracking the healthcare professional (HCP)'s ID
            # Click 'View More Details' link to access details page 
            # for that professional with the specific ID
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, f"//a[contains(@onclick,'{hcp_id}')]"))).click()
            # Scrape data from details page into a dictionary
            hcp_dict = gen_hcp_dict()
            # Convert dict to pandas dataframe (Need to set an index 
            # since we are passing scalar values)
            df_hcp_dict = pd.DataFrame(hcp_dict, index=[0])
            # Append df row directly to existing master list csv
            df_hcp_dict.to_csv(f'{file_name}', mode='a', header=False)
            # Print row that was just scraped (To track progress)
            print(f'Scraped row {index+1} of page {target_page}')
            # Update (+1) target page after successfully scrapping
            # all 10 records on page
            if index == len(id_list):
                print(f'Completed scraping for page {target_page}')
                target_page += 1
                print('Updated target page ' + str(target_page))
            else:
                pass
            # Head back to homepage by clicking 'Back to Search 
            # Results' link
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.LINK_TEXT, 'Back to Search Results'))).click() 
            # Go to latest updated target page
            locate_target_page(target_page)
    # Define steps when program reaches last page (essentially 
    # similar to the steps above)
    else:
        locate_target_page(target_page)
        print('Working on last page')
        target_page_html = driver.find_element_by_xpath("//body").get_attribute('outerHTML')
        driver.implicitly_wait(1)
        all_ids = re.findall("P[0-9]{5}[A-Z]{1}", target_page_html)
        id_list = list(dict.fromkeys(all_ids))
        for index, hcp_id in enumerate(id_list): 
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.XPATH, f"//a[contains(@onclick,'{hcp_id}')]"))).click()                 hcp_dict = gen_hcp_dict()
            df_hcp_dict = pd.DataFrame(hcp_dict, index=[0])
            df_hcp_dict.to_csv('master_list.csv', mode='a', header=False)
            print(f'Scraped row {index+1} of {target_page}')
            if index == len(id_list)-1:
                print(f'Completed scraping for page {target_page}')
                print('Mission Complete')
            else:
                pass
            WebDriverWait(driver, waittime).until(EC.presence_of_element_located((By.LINK_TEXT, 'Back to Search Results'))).click() 
            locate_target_page(target_page)
#==========================================================================================================#
